# Week 3 assignment

##### All of the marked sections are marked

In this project, we cluster the neighbourhoods of Toronto by similarity

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Import and clean the data
I made a csv file from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

It was uploaded to IMB Watson Studio and imported in the cell below as df_data_1. This has sensitive info and so will be hidden. Web scraping is completely unnecessary for this.

In [2]:
# The code was removed by Watson Studio for sharing.

In [123]:
neigh_list = df_data_1
neigh_list = neigh_list[neigh_list.Borough!='Not assigned']

for index, row in neigh_list.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']

neigh_list.head(12)



,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


# The above is one of the required dataframe and below is the shape



In [125]:
neigh_list.shape

(210, 3)

### Time to get the locations

Get The neighbourhood coordinates using geolocator. It works for most of the neighbourhoods, but we can see that it fails for a few of them. 
Because there are only a few that failed, it's feasible to just go look up the coordinated on wikipedia. 

In [124]:
from geopy.geocoders import Nominatim

missingLong = {'Humewood-Cedarvale':-79.425833,
'Caledonia-Fairbanks': -79.450443,
'CFB Toronto':-79.465556,
'Del Ray':-79.482948,
'North Midtown':-79.412439,
'Canada Post Gateway Processing Centre':-79.321783,
'Island airport':-79.396218,
'Railway Lands':-79.382412,
'Humber Bay Shores':-79.477179,
'Beaumond Heights':-79.566154,
'Stn A PO Boxes 25 The Esplanade':-79.374875,
'Business Reply Mail Processing Centre 969 Eastern':-79.321761}

missingLat = {'Humewood-Cedarvale':43.691111,
'Caledonia-Fairbanks':43.695556,
'CFB Toronto':43.743889,
'Del Ray':43.694929,
'North Midtown':43.677915,
'Canada Post Gateway Processing Centre':43.662187,
'Island airport':43.629200,
'Railway Lands':43.645041,
'Humber Bay Shores':43.627315,
'Beaumond Heights':43.734228,
'Stn A PO Boxes 25 The Esplanade':43.646393,
'Business Reply Mail Processing Centre 969 Eastern':43.662171}

geolocator = Nominatim(user_agent="Tor_explorer")
long = []
lat = []
for n in neigh_list.Neighbourhood.values:
    loc = geolocator.geocode(n+',Toronto, Ontario, Canada')
    try:
        long.append(loc.longitude)
        lat.append(loc.latitude)
    except:
        print(n + " failed")
        long.append(missingLong[n])
        lat.append(missingLat[n])

Toronto_data = neigh_list.copy()        
Toronto_data.insert(3,"Longitude",long)
Toronto_data.insert(4,"Latitude",lat)

Toronto_data.head(12)
    

Humewood-Cedarvale failed
Caledonia-Fairbanks failed
CFB Toronto failed
Del Ray failed
North Midtown failed
Canada Post Gateway Processing Centre failed
Island airport failed
Railway Lands failed
Humber Bay Shores failed
Beaumond Heights failed
Stn A PO Boxes 25 The Esplanade failed
Business Reply Mail Processing Centre 969 Eastern failed


,Postcode,Borough,Neighbourhood,Longitude,Latitude
2,M3A,North York,Parkwoods,-79.320197,43.758800
3,M4A,North York,Victoria Village,-79.311189,43.732658
4,M5A,Downtown Toronto,Harbourfront,-79.380150,43.640080
5,M6A,North York,Lawrence Heights,-79.450933,43.722778
6,M6A,North York,Lawrence Manor,-79.437507,43.722079
7,M7A,Queen's Park,Queen's Park,-79.390573,43.660609
9,M9A,Downtown Toronto,Queen's Park,-79.390573,43.660609
10,M1B,Scarborough,Rouge,-79.165837,43.804930
11,M1B,Scarborough,Malvern,-79.221701,43.809196
13,M3B,North York,Don Mills North,-79.345944,43.775347


# The above is the required dataframe


### Make a map
Check that it makes sense

In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[Toronto_data.Latitude.mean(), Toronto_data.Longitude.mean()], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Time to get the data from foursquare. Fist input hidden credentials, then create a function that formats things for us.

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print("issue on ",name, lat, lng)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto 

In [12]:
print(Toronto_venues.shape)
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))
Toronto_venues.head()

(6379, 7)
There are 338 uniques categories.


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
3,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
4,Parkwoods,43.7588,-79.320197,TD Canada Trust,43.757569,-79.314976,Bank


Now that we have the data, we can do a one hot encoding, and group by neighbourhoods and normalize so that we can apply the k mean algorithm.

In [13]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping 

In [14]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped.shape

(206, 339)

Time to apply the actual algorithm

In [88]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
pd.Series(kmeans.labels_).value_counts()


[2 0 0 0 2 3 2 0 0 0]


0    108
2     70
3     24
1      3
4      1
dtype: int64

Lets make a dataframe showing the most common venues

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Cosmetics Shop,Clothing Store,Italian Restaurant,Japanese Restaurant,Gastropub
1,Agincourt,Chinese Restaurant,Train Station,Food Court,Cantonese Restaurant,Vietnamese Restaurant,Hong Kong Restaurant,Korean Restaurant,Coffee Shop,Asian Restaurant,Shopping Mall
2,Agincourt North,Bakery,Ice Cream Shop,Chinese Restaurant,Movie Theater,Bank,Taco Place,Beer Store,Liquor Store,Sporting Goods Shop,Spa
3,Albion Gardens,Fried Chicken Joint,Video Store,Sandwich Place,Hardware Store,Fast Food Restaurant,Beer Store,Grocery Store,Pharmacy,Pizza Place,Fish & Chips Shop
4,Alderwood,Pizza Place,Gym,Pub,Dance Studio,Skating Rink,Sandwich Place,Pharmacy,Coffee Shop,Farmers Market,Empanada Restaurant


Create a map showing the clustering

In [65]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,-79.320197,43.758800,0.0,Gas Station,Pharmacy,Chinese Restaurant,Bank,Liquor Store,Laundry Service,Shopping Mall,Coffee Shop,Discount Store,Caribbean Restaurant
3,M4A,North York,Victoria Village,-79.311189,43.732658,0.0,Bus Line,Thai Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,M5A,Downtown Toronto,Harbourfront,-79.380150,43.640080,2.0,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Steakhouse,Fried Chicken Joint,Brewery,Sushi Restaurant
5,M6A,North York,Lawrence Heights,-79.450933,43.722778,0.0,Clothing Store,Coffee Shop,American Restaurant,Restaurant,Sporting Goods Shop,Jewelry Store,Cosmetics Shop,Fast Food Restaurant,Toy / Game Store,Electronics Store
6,M6A,North York,Lawrence Manor,-79.437507,43.722079,0.0,Electronics Store,Pharmacy,Park,Kids Store,Bank,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm


In [122]:
# create map
map_clusters = folium.Map(location=[Toronto_data.Latitude.mean(), Toronto_data.Longitude.mean()], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.Dark2(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    val = cluster
    if cluster != cluster:
        print("issue on ", poi)
        val = 6
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[int(val)-1],
        fill=True,
        fill_color=rainbow[int(val)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

issue on  West Deane Park
